In [5]:
%pip install ../../../

Processing /home/sjoshi/spuco
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for spuco: filename=spuco-0.0.1-py3-none-any.whl size=67905 sha256=a19bb2d119cfef957b8c8587afd22b396729df94634dc836c05e62feffd4a935
  Stored in directory: /tmp/pip-ephem-wheel-cache-9w5r27nu/wheels/ef/5d/43/a265894b1d52121a51705a208277e8d9a9670e95fa1a2e7ae6
Successfully built spuco
  Attempting uninstall: spuco
    Found existing installation: spuco 0.0.1
    Uninstalling spuco-0.0.1:
      Successfully uninstalled spuco-0.0.1
Note: you may need to restart the kernel to use updated packages.


In [6]:
import torch 

device = torch.device("cuda:7")

In [7]:
from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty
import torchvision.transforms as T

classes = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]
difficulty = SpuriousFeatureDifficulty.MAGNITUDE_EASY

trainset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    spurious_correlation_strength=0.99,
    classes=classes,
    split="train",
)
trainset.initialize()

testset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    classes=classes,
    split="test"
)
testset.initialize()

In [8]:
from spuco.models import model_factory 
from torch.optim import SGD
from spuco.end2end import LFF

bias_model=model_factory("lenet", trainset[0][0].shape, trainset.num_spurious).to(device)
model=model_factory("lenet", trainset[0][0].shape, trainset.num_spurious).to(device)
lff = LFF(
    trainset=trainset,
    bias_model=bias_model,
    debias_model=model,
    batch_size=64,
    bias_optimizer=SGD(model.parameters(), lr=1e-3, momentum=0.9, nesterov=True),
    debias_optimizer=SGD(model.parameters(), lr=1e-3, momentum=0.9, nesterov=True),
    q=1.0,
    num_epochs=5,
    device=device,
    verbose=True
)
lff.train()

AttributeError: 'LFF' object has no attribute 'biased_trainer'

In [ ]:
from spuco.evaluate import Evaluator

evaluator = Evaluator(
    testset=testset,
    group_partition=testset.group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=model,
    device=device,
    verbose=True
)
evaluator.evaluate()

In [ ]:
evaluator.worst_group_accuracy

In [ ]:
evaluator.average_accuracy

In [ ]:
evaluator.evaluate_spurious_task()